In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime

# Data import from our keylogger

In [2]:
file_path = 'keylog.txt'
columns = ['Timestamp', 'Key', 'Action']
dataframe = pd.read_csv(file_path, sep=";", names=columns, encoding = "ISO-8859-1")

display(dataframe)

,Timestamp,Key,Action
0,"2021-10-20 11:28:11,445",Key.shift,press
1,"2021-10-20 11:28:11,628",'B',press
2,"2021-10-20 11:28:11,717",'B',release
3,"2021-10-20 11:28:11,750",Key.shift,release
4,"2021-10-20 11:28:11,894",'o',press
5,"2021-10-20 11:28:11,990",'n',press
6,"2021-10-20 11:28:12,054",'o',release
7,"2021-10-20 11:28:12,118",'n',release
8,"2021-10-20 11:28:12,201",'j',press
9,"2021-10-20 11:28:12,297",'o',press


# Preprocessing

In the event that a key is pressed for a 'long time' (at least 2 consecutive row with the same 'Key' value and 'Action' = press), we only keep the first pressed

In [3]:
for i in range(dataframe.index.start, dataframe.index.stop):
    current_key = dataframe['Key'][i]
    if dataframe['Action'][i] == 'press':
        press = True
        j = i + 1
        while press:
            if j < dataframe.index.stop and dataframe['Action'][j] == 'press' and current_key == dataframe['Key'][j]:
                dataframe = dataframe.drop([j])
                dataframe = dataframe.reset_index(drop=True)
            else:
                break
                
display(dataframe)

,Timestamp,Key,Action
0,"2021-10-20 11:28:11,445",Key.shift,press
1,"2021-10-20 11:28:11,628",'B',press
2,"2021-10-20 11:28:11,717",'B',release
3,"2021-10-20 11:28:11,750",Key.shift,release
4,"2021-10-20 11:28:11,894",'o',press
5,"2021-10-20 11:28:11,990",'n',press
6,"2021-10-20 11:28:12,054",'o',release
7,"2021-10-20 11:28:12,118",'n',release
8,"2021-10-20 11:28:12,201",'j',press
9,"2021-10-20 11:28:12,297",'o',press


In [4]:
dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S,%f')

## Details

**columns_nd** = *nd for normalize data* <br>
**Hold Duration (HD)** = *Duration between press and release of the same key* <br>
**Hold time in digraph (HTD)** = *Dwell time with the context of the preceding keystroke* <br/>
**Cumulative Dwell Time (CDT)** = *Holding time for two consecutive keys without flying time* <br>
**Digraph latency (DL)** = *Flight time between two keystrokes*





In [5]:
columns_nd = ['Key1', 'Key2', 'HD', 'HTD', 'CDT', 'DL']# 'Shift', 'Alt', 'Ctrl', 'Win']
df = pd.DataFrame(columns = columns_nd)

def search_for_press(df):
    for i in range(df.index.start, df.index.stop):
        if df['Action'][i] == 'press':
            return i

def search_for_release(df, press_index):
    for i in range (press_index, df.index.stop):
        if df['Key'][i] == df['Key'][press_index] and df['Action'][i] == 'release':
            return i
        
def search_for_following_key(df, key_index):
    for i in range(key_index, df.index.stop):
        if df['Key'][i] != df['Key'][key_index] and df['Action'][i] == 'press':
            return i
  


while dataframe.empty == False:
    end = False
    press_index = search_for_press(dataframe)
    release_index = search_for_release(dataframe, press_index)
    if dataframe.index.stop-dataframe.index.start == 2:
        end = True
    if end == False:
        following_key_press_index = search_for_following_key(dataframe, press_index)
        following_key_release_index = search_for_release(dataframe, following_key_press_index)
        Key1 = dataframe['Key'][press_index]
        Key2 = dataframe['Key'][following_key_press_index]
        HD = dataframe['Timestamp'][release_index] - dataframe['Timestamp'][press_index]
        HD = HD.total_seconds()
        HTD = dataframe['Timestamp'][following_key_release_index] - dataframe['Timestamp'][following_key_press_index]
        HTD = HTD.total_seconds()
        display
        CDT = HD + HTD
        DL = dataframe['Timestamp'][following_key_press_index] - dataframe['Timestamp'][release_index]
        DL = DL.total_seconds()
        dataframe = dataframe.drop([press_index, release_index])
        dataframe = dataframe.reset_index(drop=True)
        tmp = pd.DataFrame([[Key1, Key2, HD, HTD, CDT, DL]], columns = columns_nd)
        df = df.append(tmp)
    else:
        Key1 = dataframe['Key'][press_index]
        HD = dataframe['Timestamp'][release_index] - dataframe['Timestamp'][press_index]
        HD = HD.total_seconds()
        HTD = None
        CDT = None
        DL = None
        dataframe = dataframe.drop([press_index, release_index])
        tmp = pd.DataFrame([[Key1, None, HD, HTD, CDT, DL]], columns = columns_nd)
        df = df.append(tmp)

        
df = df.reset_index(drop=True)
       
display(df)
    



,Key1,Key2,HD,HTD,CDT,DL
0,Key.shift,'B',0.305,0.089,0.394,-0.122
1,'B','o',0.089,0.16,0.249,0.177
2,'o','n',0.160,0.128,0.288,-0.064
3,'n','j',0.128,0.132,0.26,0.083
4,'j','o',0.132,0.127,0.259,-0.036
5,'o','u',0.127,0.092,0.219,-0.03
6,'u','r',0.092,0.092,0.184,0.001
7,'r',None,0.092,None,None,None


# Metrics representation